# supervisor image labelling code
Hello lovely supervisors, thank you for doing this second labelling of images for me!

### how does this code work?
You should be able to just start labelling, running each cell should execute everything in the correct order.
<br>
In your respective folders will be \<name>_labelling_0.csv, containing the list of images (with no labels) and a zip file of images.
<br>
In the labelling cell, an image should appear with input box, in which you write the label. Unfortunately I am not skilled enough to make a back / forward button, so you will have one go at labelling the image (or email me).
<br>
You can stop labelling at any time, by writing 'stop' then come back to it.
### importantly!
* I don't think binder saves the progress, so once you have finished with a set of labelling, download the csv locally, then upload to your respective folder once you start again.
### once finished labelling
\<name>_labelling_n.csv with the largest number is the final csv I need, then just email me it :)

In [ ]:
## imports
import numpy as np
import pandas as pd
import os
from glob import glob
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output
import shutil
from PIL import Image

In [2]:
## please enter your name ! 
supervisor_name =  #'Alan' 'Declan' 'Paul' 


In [ ]:
## some set up 
if supervisor_name == 'Alan':
    path = './alan_sample/'
    image_file = 'alan_sample_images'
elif supervisor_name == 'Declan':
    path = './dec_sample/'
    image_file = 'dec_sample_images'  
elif supervisor_name == 'Paul':
    path = './paul_sample/'
    image_file = 'paul_sample_images'
else:
    print('please type your first name!')

## csv handelling
all_csvs = glob(path+'*.csv')
try: 
  supervisor_csv_str = all_csvs[0]
except IndexError:
  supervisor_csv_str = all_csvs


base_csv_name = supervisor_csv_str[:-5]
number_csvs = len(all_csvs)
csv_full_path = f'{base_csv_name}{int(number_csvs)-1}.csv'
full_save_csv_path = f'{base_csv_name}{number_csvs}.csv'
csv_read_pd = pd.read_csv(csv_full_path)

n_images_done = csv_read_pd['number_label'].count()

## image folder handelling
# check if folder for image labelling has been unzipped
folder_path = path+image_file+'/'
if not os.path.exists(folder_path):
    # it has not been unzipped, will do that now
    shutil.unpack_archive(path+image_file+'.zip', path+image_file)
    print(f'unzipped {image_file} sucessfully!')


## stuff to print
print('!! PLEASE CHECK THIS INFO !!')
print('(they should have different names)')
print(f'csv reading in: {csv_full_path}')
print(f'csv saving to: {full_save_csv_path}')
print(f'number of images already labelled {n_images_done}')
print('! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ')


In [ ]:
## image labelling cell
### function to ensure all images are captured + save the new csv
def saving_function(new_images, new_labels, og_df_name_list):
    if len(new_images) < len(og_df_name_list):
        not_labelled = list(set(og_df_name_list) - set(new_images))
        new_images.extend(not_labelled)  # Modifies the list in place
        new_labels.extend([np.nan] * len(not_labelled))

    new_csv_dic = {'image_name': new_images, 'number_label':new_labels}
    df = pd.DataFrame(new_csv_dic)
    df.to_csv(full_save_csv_path, index=False)
    print(f'{full_save_csv_path} saved!')
    return df

# some things need to be defined before starting the processing loop
image_name_lst = []
new_image_label_lst = []
all_image_names = list(csv_read_pd['image_name'])

with open(csv_full_path, mode='r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)  # Read the header row
    # loop through each row
    for row in csv_reader:
        image_name = row[0]
        label = row[1]        
        
        # label returns false for nan, so will just write names / labels aleady there
        if label: 
            #num = int(label) # convert to number (corresponding to label)
            image_name_lst.append(image_name)
            new_image_label_lst.append(label)
                
        ## if is a nan, will follow this loop for labelling
        else:
            print(image_name)
            im = Image.open(os.path.join(folder_path, image_name + '.png'))
            plt.figure(figsize=(8, 8))
            plt.imshow(im)
            plt.axis('off')
            plt.show()
            while True: # loop will continue until a valid entry is passed
                inp = input("Enter a label (0-9 or 11, or 'stop' to quit): ") ## label of image

                ## if write 'stop' will stop labelling + save what has been done
                if inp.lower() == 'stop':
                    print("Stopped labeling.")
                    done_images = saving_function(image_name_lst,new_image_label_lst,all_image_names)
                    raise SystemExit  # Exit gracefully
                
                try:
                    num = int(inp) # convert to number (corresponding to label)
                    if 0 <= num <= 11:  # Check if the number is in the valid range
                        clear_output(wait=True) # get rid of image once you've labelled it
                        image_name_lst.append(image_name)
                        new_image_label_lst.append(num)
                        break  # Exit the loop if the input is valid
                    else:
                        print("Error: Label must be between 0 and 11.")
                except ValueError: # if we don't input a number -> stop labelling, so will break
                    print("not words, what are you doing???")
                                        
done_images = saving_function(image_name_lst,new_image_label_lst,all_image_names)
print('Finished labelling! thanks!')